In [4]:
import pandas as pd
import statsmodels.formula.api as smf
import re
import matplotlib.pyplot as plt
import re

In [8]:
df = pd.read_csv('/Users/marcomontenegro/Desktop/MSQE PROJECT/Project_Data/financial_data_returns_2018_2024.csv', index_col=0, parse_dates=True)

# Helper: Safe clean and matching
def clean_col(col):
    # Lowercase, remove non-alphanumeric except underscores, strip repeated/leading/trailing underscores
    s = re.sub(r'[^0-9a-zA-Z_]+', '_', col).strip('_')
    s = re.sub(r'_+', '_', s)
    return s.lower()

df_clean = df.copy()
df_clean.columns = [clean_col(col) for col in df_clean.columns]
print("Cleaned columns:", list(df_clean.columns))

def find_col(substring, columns):
    substring = substring.lower().strip('_')
    for col in columns:
        if substring in col.strip('_').lower():
            return col
    return None

icln_ret_col = find_col('icln_ret', df_clean.columns)
xle_ret_col = find_col('xle_ret', df_clean.columns)
carbon_shock_col = find_col('carbon_shock', df_clean.columns)
vix_col = find_col('vix', df_clean.columns)
brent_col = find_col('brent', df_clean.columns)
sp500_ret_col = find_col('sp500_ret', df_clean.columns)
dgs10_col = find_col('dgs10', df_clean.columns)
dgs10_diff_col = find_col('dgs10_diff', df_clean.columns)

# Print detected
print("\nDetected columns:")
for varname, val in [
    ("icln_ret_col", icln_ret_col),
    ("xle_ret_col", xle_ret_col),
    ("carbon_shock_col", carbon_shock_col),
    ("vix_col", vix_col),
    ("brent_col", brent_col),
    ("sp500_ret_col", sp500_ret_col),
    ("dgs10_col", dgs10_col),
    ("dgs10_diff_col", dgs10_diff_col)
]:
    print(f"{varname}: {val}")

# Controls: Only use if numeric, not all NA or constant
controls = []
for col in [vix_col, brent_col, sp500_ret_col, dgs10_col]:
    if col and col in df_clean.columns:
        ser = df_clean[col]
        if pd.api.types.is_numeric_dtype(ser) and ser.notnull().sum() > 1 and ser.nunique(dropna=True) > 1:
            controls.append(col)
        else:
            print(f"Skipped control '{col}': Not numeric, too many NAs, or constant")

print("\nFinal controls used:", controls)

# --- Regression setup ---
regressions = []
if icln_ret_col and carbon_shock_col:
    formula_icln = f"{icln_ret_col} ~ {carbon_shock_col}" + (" + " + " + ".join(controls) if controls else "")
    needed_cols_icln = [icln_ret_col, carbon_shock_col] + controls
    regdata_icln = df_clean[needed_cols_icln].dropna()
    regressions.append(('ICLN', formula_icln, regdata_icln))
else:
    print("ICLN regression cannot run: missing required columns.")

if xle_ret_col and carbon_shock_col:
    formula_xle = f"{xle_ret_col} ~ {carbon_shock_col}" + (" + " + " + ".join(controls) if controls else "")
    needed_cols_xle = [xle_ret_col, carbon_shock_col] + controls
    regdata_xle = df_clean[needed_cols_xle].dropna()
    regressions.append(('XLE', formula_xle, regdata_xle))
else:
    print("XLE regression cannot run: missing required columns.")

# --- Run regressions ---
for label, formula, data in regressions:
    print(f"\n{label} Regression formula: {formula}")
    print(f"N rows used: {len(data)}")
    model = smf.ols(formula=formula, data=data).fit(cov_type='HC1')
    print(f"\n==== Regression Results: {label} ====\n")
    print(model.summary())


print("\nDone.")


Cleaned columns: ['icln_icln', 'xle_xle', 'vix_vix', 'brent_bz_f', 'sp500_gspc', 'dgs10', 'carbon_eua', 'icln_icln_ret', 'xle_xle_ret', 'brent_bz_f_ret', 'carbon_eua_ret', 'sp500_gspc_ret', 'dgs10_diff', 'carbon_shock_resid', 'carbon_shock']

Detected columns:
icln_ret_col: icln_icln_ret
xle_ret_col: xle_xle_ret
carbon_shock_col: carbon_shock_resid
vix_col: vix_vix
brent_col: brent_bz_f
sp500_ret_col: None
dgs10_col: dgs10
dgs10_diff_col: dgs10_diff

Final controls used: ['vix_vix', 'brent_bz_f', 'dgs10']

ICLN Regression formula: icln_icln_ret ~ carbon_shock_resid + vix_vix + brent_bz_f + dgs10
N rows used: 1763

==== Regression Results: ICLN ====

                            OLS Regression Results                            
Dep. Variable:          icln_icln_ret   R-squared:                       0.038
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     7.262
Date:                Sun, 

In [5]:
# Load data (should already have rolling vol, carbon_shock, returns, macro)
df = pd.read_csv('/Users/marcomontenegro/Desktop/MSQE PROJECT/Project_Data/financial_data_returns_shocks_vol_2018_2024.csv', index_col=0, parse_dates=True)

def clean_col(col):
    s = re.sub(r'[^0-9a-zA-Z_]+', '_', col).strip('_')
    s = re.sub(r'_+', '_', s)
    return s.lower()

df_clean = df.copy()
df_clean.columns = [clean_col(col) for col in df_clean.columns]
print("Cleaned columns:", list(df_clean.columns))

def find_col(substring, columns):
    substring = substring.lower().strip('_')
    for col in columns:
        if substring in col.strip('_').lower():
            return col
    return None

# Key columns for both ETFs
icln_vol_col = find_col('icln_vol', df_clean.columns)
xle_vol_col  = find_col('xle_vol', df_clean.columns)
carbon_shock_col = find_col('carbon_shock', df_clean.columns)
vix_col = find_col('vix', df_clean.columns)
brent_col = find_col('brent', df_clean.columns)
sp500_ret_col = find_col('sp500_ret', df_clean.columns)
dgs10_col = find_col('dgs10', df_clean.columns)
dgs10_diff_col = find_col('dgs10_diff', df_clean.columns)

# Print detected
print("\nDetected columns:")
for varname, val in [
    ("icln_vol_col", icln_vol_col),
    ("xle_vol_col", xle_vol_col),
    ("carbon_shock_col", carbon_shock_col),
    ("vix_col", vix_col),
    ("brent_col", brent_col),
    ("sp500_ret_col", sp500_ret_col),
    ("dgs10_col", dgs10_col),
    ("dgs10_diff_col", dgs10_diff_col)
]:
    print(f"{varname}: {val}")

# Controls: Only use if numeric, not all NA or constant
controls = []
for col in [vix_col, brent_col, sp500_ret_col, dgs10_col]:
    if col and col in df_clean.columns:
        ser = df_clean[col]
        if pd.api.types.is_numeric_dtype(ser) and ser.notnull().sum() > 1 and ser.nunique(dropna=True) > 1:
            controls.append(col)
        else:
            print(f"Skipped control '{col}': Not numeric, too many NAs, or constant")

print("\nFinal controls used:", controls)

# --- Regression setup ---
regressions = []
if icln_vol_col and carbon_shock_col:
    formula_icln = f"{icln_vol_col} ~ {carbon_shock_col}" + (" + " + " + ".join(controls) if controls else "")
    needed_cols_icln = [icln_vol_col, carbon_shock_col] + controls
    regdata_icln = df_clean[needed_cols_icln].dropna()
    regressions.append(('ICLN_vol', formula_icln, regdata_icln))
else:
    print("ICLN volatility regression cannot run: missing required columns.")

if xle_vol_col and carbon_shock_col:
    formula_xle = f"{xle_vol_col} ~ {carbon_shock_col}" + (" + " + " + ".join(controls) if controls else "")
    needed_cols_xle = [xle_vol_col, carbon_shock_col] + controls
    regdata_xle = df_clean[needed_cols_xle].dropna()
    regressions.append(('XLE_vol', formula_xle, regdata_xle))
else:
    print("XLE volatility regression cannot run: missing required columns.")

# --- Run regressions ---
for label, formula, data in regressions:
    print(f"\n{label} Regression formula: {formula}")
    print(f"N rows used: {len(data)}")
    model = smf.ols(formula=formula, data=data).fit(cov_type='HC1')
    print(f"\n==== Regression Results: {label} ====\n")
    print(model.summary())

print("\nDone.")

# ------------------------------------------------------------

Cleaned columns: ['icln_icln', 'xle_xle', 'vix_vix', 'brent_bz_f', 'sp500_gspc', 'dgs10', 'carbon_eua', 'icln_icln_ret', 'xle_xle_ret', 'brent_bz_f_ret', 'carbon_eua_ret', 'sp500_gspc_ret', 'dgs10_diff', 'carbon_shock_resid', 'carbon_shock', 'icln_vol', 'xle_vol']

Detected columns:
icln_vol_col: icln_vol
xle_vol_col: xle_vol
carbon_shock_col: carbon_shock_resid
vix_col: vix_vix
brent_col: brent_bz_f
sp500_ret_col: None
dgs10_col: dgs10
dgs10_diff_col: dgs10_diff

Final controls used: ['vix_vix', 'brent_bz_f', 'dgs10']

ICLN_vol Regression formula: icln_vol ~ carbon_shock_resid + vix_vix + brent_bz_f + dgs10
N rows used: 1754

==== Regression Results: ICLN_vol ====

                            OLS Regression Results                            
Dep. Variable:               icln_vol   R-squared:                       0.549
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     195.0
Date:    